In [ ]:
pip install keras tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 257.2 kB/s eta 0:00:000:00:010:00:01:01
     ━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.8/589.8 MB 183.2 kB/s eta 0:49:20:24